In [7]:
import pandas as pd
import json
from collections import defaultdict
import re
import dataExtraction2

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados. PRUEBA

Si se han usado los pasos minimos:

In [8]:
def porDefecto():
    return defaultdict(lambda: {"tarjetasCreadas" : 0, "tarjetasMovidas" : 0, "tarjetasEliminadas" : 0})

def extraerDatosTarjetas(rawData):
    erGameObject = re.compile(r'\bgame-object$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    
    data = defaultdict(porDefecto)

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erGameObject.search(obj) : #Si el objeto de la acción es un objecto
            name = evento["actor"]["name"]

            if erInteracted.search(verb):
                try:
                    levelCode = evento["result"]["extensions"]["level"]
                    dataExtraction2.anadirAlDiccionario(data, evento, name, levelCode)
                except KeyError:
                    pass

    return data            

In [9]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data = extraerDatosTarjetas(rawData)
pd.DataFrame(data)

JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
#pd.DataFrame(data)

pd.DataFrame(data)

,gqoj,hgfx,uala,qeqyj,kmqou,ikcar,eprdu,gcpze,loemd,pbgky,ftpnb,fkmtp,ejeuw,khadt,gleqe,srjon,xnpqf,uzzpo,vsxld,ipdcp,epqkm,zxvkj,sxipu,hjpbw,zwzfv,bdwyj,pygoc,oahey,yfohf,kkoif,krahz,yvpmf,ubmtx,tjfzy,asopk,tekwp,cmbfi,rdiaf,onyep,zcibf,auvqa,bhyhw,dunul,edxmq,dwqwi,swvxx,grkij,nmveq,accul,sjpkc,ewqbs,ucdvy,gguuc,yxewz,dvmlu,tinco,psido,wjaxi,mjiso,zbgwk,hpdxj,fivin,yxjji,tgwmn,fyyya,hqmhq,mymho,camhi,gdljy,dmwvy,lwryl,ztyed,kkliq,rziia,ngavz,aotlm,knewj,huyyb,qfsdj,tqgyp,mgaak,atlzs,stiry,tavin,vvqlm,vddlg,zdnso,bjvnt,lhrbj,yazod,idmzi,xtdrq,jhbuv,rqlug,tvxow,wqmgq,jozyx,ahxym,pbwkf,sbexy,fmtrq,gzgfk,khfha,yyvas,itkas,buvgs,nuasx,ptmkr,icllt,rkmrw,thaho,pzfdd,mewpz,uniee,rqaby,kzles,qwnpf,hcbyl,nhbnj,esrit,nplrf,qbsjv,blvkx,rfofs,xbmmw,orjyn,fbldj,prpej,eayrh,lohyo,pqwaf,ixrux,qgcgc,cojqb,raqdt,fmupf,rsjxu,sybwc,pleui,tflzv,bajbm,tcvyz,vkfnt,olugf,epjcx
editor_level,"[{'tarjetasCreadas': 7, 'tarjetasMovidas': 7, 'tarjetasEliminadas': 0}]",NaN,NaN,NaN,NaN,NaN,NaN,"[{'tarjetasCreadas': 3, 'tarjetasMovidas': 1, 'tarjetasEliminadas': 0}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'tarjetasCreadas': 3, 'tarjetasMovidas': 2, 'tarjetasEliminadas': 0}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
variables_3,"[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]",NaN,NaN,"[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 1, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 2, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 2, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 1, 'tarjetasEliminadas': 0}]","[{'tarjetasCreadas': 1, 'tarjetasMovidas': 0, 'tarjetasEliminadas': 0}]","[{'tarjetasCre